In [2]:
import pandas as pd
from pdfquery import PDFQuery
import re

In [3]:
pdf = PDFQuery("resultado1570777439.pdf")
pdf.load()

In [20]:
textos = "\n".join(pdf.tree.xpath('//LTTextLineHorizontal/text()'))
with open("teste.txt", "wb") as file:
    file.write(textos.encode())